In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import gc
from projectetl.utils.dataload import (load_survey_data_do, save_survey_with_pickle,
                                       load_survey_from_pickle)
from projectetl.utils.config import (ENCFT_SURVEY_PATH, ENCFT_PREVIOUS_SURVEY_PATH,
                                     ENCFT_OBJECT_DIR, INFLATION_OBJECT_DIR, DATA_DIR,
                                     ENCFT_DIR)
from criteriaetl.utils.expansion_func import (get_value_counts_with_expansion_factor,
    get_percentage_table_with_expansion_factor, get_weighted_average)

In [ ]:
get_raw_incomes = 0

In [ ]:
if get_raw_incomes:
    print('loading csv')
    survey_incomes = load_survey_data_do(ENCFT_DIR / '2020 con pobreza calculada.csv', load_func=lambda path: pd.read_csv(str(path), low_memory=False))
    save_survey_with_pickle(survey_incomes, ENCFT_OBJECT_DIR / 'encft2020.pkl')
else:
    print('loading pickle')
    survey_incomes = load_survey_from_pickle(ENCFT_OBJECT_DIR / 'encft2020.pkl')
    
key_member = ['trimestre', 'id_hogar', 'id_persona']
cols_from_incomes = key_member + ['ingpercapitadef', 'lpextrema', 'lpgeneral', 'quedateencasa', 
                                  'trsoc_def', 'trsoc', 'factor_expansion_anual']
survey_incomes_slim = survey_incomes[cols_from_incomes].copy()
del survey_incomes
gc.collect()

In [ ]:
survey_incomes_slim['pobreza'] = survey_incomes_slim.ingpercapitadef.le(survey_incomes_slim.lpgeneral)
survey_incomes_slim['pobreza_extrema'] = survey_incomes_slim.ingpercapitadef.le(survey_incomes_slim.lpextrema)


In [ ]:
households_poverty = survey_incomes_slim.groupby(key_member[:-1])[['pobreza', 'pobreza_extrema', 'factor_expansion_anual']].first()

In [ ]:
get_percentage_table_with_expansion_factor(survey_incomes_slim, 'pobreza', 'factor_expansion_anual')

In [ ]:
get_percentage_table_with_expansion_factor(households_poverty, 'pobreza', 'factor_expansion_anual')

In [ ]:
get_percentage_table_with_expansion_factor(survey_incomes_slim, 'pobreza_extrema', 'factor_expansion_anual')

In [ ]:
get_percentage_table_with_expansion_factor(households_poverty, 'pobreza_extrema', 'factor_expansion_anual')

In [ ]:
survey_incomes_slim.quedateencasa.value_counts()

In [ ]:
survey_incomes_slim.groupby(['trimestre', 'id_hogar'])['quedateencasa'].sum().value_counts()